In [8]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler
from IPython.display import display
from IPython.core.debugger import set_trace

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. 초기화

In [9]:
crawler = NewsCrawler(storage='bigquery')

checking bigquery storage... done (25.71 seconds)
we have total 339967 articles (310905 downloaded, 29062 trashed)


# 1. URL을 모은다

In [10]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [11]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (94.30 seconds): Collecting URLs... euronews            

100.00% (0.09 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,25765,25695,671
investing.com,936,936,95
reuters,1120,1120,94
euronews,877,877,64
yahoo,394,394,55
...,...,...,...
afr,456,456,0
axios,131,131,0
cbs,417,417,0
nationalreview,704,702,0


,url,pubs,actual_pub
0,https://politico.com/magazine/story/2019/10/23...,"politico, realclearpolitics",politico
1,https://politico.com/magazine/story/2019/10/21...,"politico, realclearpolitics",politico
2,https://politico.com/news/2019/10/30/kamala-ha...,"politico, realclearpolitics",politico
3,https://politico.com/magazine/story/2019/10/30...,"politico, realclearpolitics",politico
4,https://politico.com/news/2019/10/31/trump-fed...,"politico, realclearpolitics",politico
...,...,...,...
65,https://nbcnews.com/nightly-news/video/katie-h...,"msnbc, nbcnews",nbcnews
66,https://nbcnews.com/news/latino/does-congress-...,"msnbc, nbcnews",nbcnews
67,https://msnbc.com/the-last-word/watch/trump-s-...,"msnbc, nbcnews",msnbc
68,https://nbcnews.com/tech/tech-news/jack-dorsey...,"msnbc, nbcnews",nbcnews


# 2. 해당 URL의 뉴스를 크롤링한다

In [12]:
crawl_result = crawler.crawl(); crawl_result

8.64% (13.12 seconds): Crawling... techcrunch          

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


24.14% (32.60 seconds): Crawling... yahoo               

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100.00% (221.36 seconds): Crawling... yahoo               



collect                                crawl              
               collected collected_unique selected downloaded trashed total
all                25765            25695      671        362     309   671
investing.com        936              936       95         28      67    95
reuters             1120             1120       94         44      50    94
euronews             877              877       64          5      59    64
yahoo                394              394       55         32      23    55
...                  ...              ...      ...        ...     ...   ...
afr                  456              456        0          0       0     0
axios                131              131        0          0       0     0
cbs                  417              417        0          0       0     0
nationalreview       704              702        0          0       0     0
cryptonews           331              331        0          0       0     0

[74 rows x 6 columns]

# 3. 뉴스를 저장한다

In [13]:
crawler.record()

Recording to bigquery storage... done (21.00 seconds)


# 4. 결과보고

In [14]:
crawler.report()

In [66]:
from pathlib import Path
downloaded = [p.stem for p in Path('newsdata/downloaded').glob('**/*.json')]
trashed = [p.stem for p in Path('newsdata/trashed').glob('**/*.json')]

In [65]:
for path in list(Path('newsdata/downloaded').glob('**/*.json'))[:10000]:
    path.unlink()

In [53]:
from pathlib import Path
import json
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.header import Header

In [54]:
acc_fname = 'smtp_gmail.json'
acc = json.loads(Path(acc_fname).read_text()); acc

{'host': 'smtp.gmail.com',
 'from': 'gem763@gmail.com',
 'to': ['gem763@gmail.com', 'sejin.kang@hanwha.com'],
 'pw': 'kkangse1'}

In [55]:
msg = MIMEMultipart('alternative')
msg['Subject'] = Header('NEWS CRAWLING REPORT', 'utf-8')
msg['From'] = acc['from']
msg['To'] = ', '.join(acc['to'])
msg.attach(MIMEText(crawler.crawl_summary.to_html(), 'html'))

with smtplib.SMTP_SSL(acc['host']) as smtp:
    smtp.login(acc['from'], acc['pw'])
    smtp.send_message(msg)